# Hopfield Networks 

In [161]:
import numpy as np
from numpy.random import randint
from numpy import exp

In [99]:
# States of the network
S = np.array([[1, 0, 1, 0, 0]], dtype=float).T

# values of connections in the nework
wvalues = np.array([0, -4, 3, 2, 0, # from state 0 to states 0, 1, 2, 3, 4
                    0, 0, 3, 3,     # from state 1 to states 1, 2, 3, 4
                    0, -1, 0,       # from state 2 to states 2, 3, 4
                    0, -1,          # from state 3 to states 3, 4
                    0])             # from state 4 to state  4

# Weight matrix
W = np.zeros((5,5))
W[np.triu_indices(5)] = wvalues

# Transition matrix for states
S_conn = np.triu(S @ S.T)
S_conn[np.diag_indices(5)] = 0

In [174]:
class Hopfield:
    def __init__(self, states, weights):
        self.states = states
        self.num_states = len(states)
        self.weights = weights
        self.states_conn = self.connections(states)
    
    def connections(self, states):
        S_conn = np.triu(self.states @ self.states.T)
        S_conn[np.diag_indices(self.num_states)] = 0
        return S_conn

In [175]:
h = Hopfield(S, W)

In [176]:
h.states_conn

array([[ 0.,  0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.]])

In [135]:
# Current state of the network
np.sum(S_conn * W)

3.0

In [139]:
# Value above 0 -> off
target = 1
S.T @ W[:,target]
S[target] = 0

np.sum(S_conn * W)

3.0

In [148]:
# Value above 0 -> on
target = 2
S.T @ W[:,target]
S[target] = 0

np.sum(S_conn * W)

3.0

In [156]:
# Value above 0 -> on; state changes, total energy goes down
target = 3
delta_state = 1 if S.T @ W[:,target] > 0 else 0
S[target] = delta_state
S_conn = np.triu(S @ S.T)

np.sum(S_conn * W)

2.0

If $\Delta E = -3$, then $P(s=1)$ increases when $T$ increases.

Recall: 
$$
    P(s_i=1) = \frac{1}{1 + e^{-\Delta E_i / T}}
$$

In [169]:
p = np.vectorize(lambda deltaE, T: 1 / (1 + exp(-deltaE / T)))
p(-3, [1,2,3,4,5])

array([ 0.04742587,  0.18242552,  0.26894142,  0.3208213 ,  0.35434369])